In [1]:
import cv2
import mediapipe as mp
import numpy as np

def cart2sph(x, y, z):
    r = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arccos(np.clip(z / r, -1.0, 1.0)) if r != 0 else 0
    phi = np.arctan2(y, x)
    return r, theta, phi


mp_face_detection = mp.solutions.face_detection.FaceDetection()
mp_face_mesh = mp.solutions.face_mesh.FaceMesh(
    static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)

cap = cv2.VideoCapture(0)
cam_pos = np.array([0, 5, 5], dtype=float)
cam_dir = np.array([0, 1, 0], dtype=float)
cam_dir /= np.linalg.norm(cam_dir)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    results = mp_face_detection.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    if results.detections:
        relative_bounding_box = results.detections[0].location_data.relative_bounding_box
        x, y, w, h = int(relative_bounding_box.xmin * frame.shape[1]), int(relative_bounding_box.ymin * frame.shape[0]), \
            int(relative_bounding_box.width * frame.shape[1]), int(relative_bounding_box.height * frame.shape[0])

        # 얼굴 중심 계산
        cx, cy = int(x + w / 2), int(y + h / 2)

        # 얼굴 방향 계산
        pose_results = mp_face_mesh.process(frame)
        if pose_results.multi_face_landmarks:
            pose_landmarks = pose_results.multi_face_landmarks[0]
            face_dir = np.array([pose_landmarks.landmark[33].x - pose_landmarks.landmark[2].x,
                                 pose_landmarks.landmark[33].y - pose_landmarks.landmark[2].y,
                                 pose_landmarks.landmark[33].z - pose_landmarks.landmark[2].z], dtype=float)
            face_dir /= np.linalg.norm(face_dir)
            
            # 카메라와 얼굴의 방향 벡터를 외부 좌표계로 변환
            R = np.array([[0, 0, 1], [1, 0, 0], [0, 1, 0]])
            #카메라가 고정되어있으므로 위치 변환 X
            #cam_pos = np.dot(R, cam_pos)
            #cam_dir = np.dot(R, cam_dir)
            face_dir = np.dot(R, face_dir)
            # 카메라와 얼굴의 방향 벡터를 구면 좌표계로 변환
            cam_r, cam_theta, cam_phi = cart2sph(*cam_dir)
            face_r, face_theta, face_phi = cart2sph(*face_dir)

            # 값 보정
            if face_phi > np.pi / 2 and cam_phi < -np.pi / 2:
                cam_phi += 2 * np.pi
            elif cam_phi > np.pi / 2 and face_phi < -np.pi / 2:
                face_phi += 2 * np.pi

            cv2.arrowedLine(frame, (cx, cy), (cx + int(50 * cam_dir[0]), cy - int(50 * cam_dir[1])), (255, 0, 0), 2)
            cv2.arrowedLine(frame, (cx, cy), (cx + int(50 * face_dir[0]), cy - int(50 * face_dir[1])), (0, 0, 255), 2)

            # 결과 출력
            cv2.putText(frame, f"Camera: r={cam_r:.2f}, theta={cam_theta * 180/np.pi:.2f}°, phi={cam_phi * 180/np.pi:.2f}°", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(frame, f"Face: r={face_r:.2f}, theta={face_theta * 180/np.pi:.2f}°, phi={face_phi * 180/np.pi:.2f}°", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA)

    cv2.imshow("Face Detection", frame)
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
